# PRE PROCESS
.JPG TO .NPY

In [1]:
# import os
# import cv2
# import numpy as np

# def jpg_to_npy_in_folders(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)
#     for item in os.listdir(input_folder):
#         item_path = os.path.join(input_folder, item)
#         if os.path.isdir(item_path):
#             jpg_to_npy_in_folders(item_path, output_folder)
#         elif os.path.isfile(item_path) and item.endswith(".jpg"):
#             img = cv2.imread(item_path)
#             img_array = np.array(img)
#             output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
#             np.save(output_file_path, img_array)

# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)
# jpg_to_npy_in_folders("/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [2]:
import os
import numpy as np
directory = '/kaggle/input/better-preproc/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)


5121

In [3]:
mildDem = [i for i in files if "mildDem" in i]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

2509 52 2560 1792


In [4]:
import imgaug.augmenters as iaa

# Define the augmenter
augmenter = iaa.Sequential([
    iaa.Multiply((0.8, 1.2)),  
    iaa.LinearContrast((0.8, 1.2)),  
])

augmented_images = []

for path in moderateDem:
    path = os.path.join("/kaggle/input/better-preproc/train", path)
    image = np.load(path)
    
    for _ in range(30):
        augmented_image = augmenter.augment_image(image)
        augmented_images.append(augmented_image)
augmented_images = np.array(augmented_images)

In [5]:
len(augmented_images)

1560

In [6]:
import cv2
import os


output_directory = "/kaggle/working"

os.makedirs(output_directory, exist_ok=True)

for i, image in enumerate(augmented_images):
    output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
    np.save(output_path, image)
    

print("Augmented images saved successfully.")


Augmented images saved successfully.
